# Improve Atmospheric Forcing Weights File

Transfer the values from a `met_gem_weight.nc`
created by `NEMO_EastCoast/NEMO_Preparation/4_weights_ATMOS/get_weight_nemo`
into a netCDF4 file with zlib compression on variables
and CF-1.6 conventions conformant attributes.

## 1. Setup

In [1]:
from datetime import datetime
import os
from pathlib import Path

import netCDF4 as nc
import numpy as np

from salishsea_tools import nc_tools

The input file is the `met_gem_weight.nc` file produced by
`NEMO_EastCoast/NEMO_Preparation/4_weights_ATMOS/get_weight_nemo`.
See the
[Atmospheric Forcing - Interolation Weights](http://salishsea-meopar-docs.readthedocs.org/en/latest/code-notes/salishsea-nemo/nemo-forcing/atmospheric.html#interpolation-weights)
docs for details of how to generate it.
The only thing you should need to change here is the path to the
`met_gem_weight.nc` file you want to work with.

In [2]:
met_gem_weight = Path('/data/dlatorne/MEOPAR/grid', 'met_gem_weight.nc')

The output file should be version controlled in the `grid/' repo.

In [3]:
netcdf4_weight = Path(
    '/data/dlatorne/MEOPAR/grid', 
    'weights-continental2.5-hrdps_202108_23feb23onward.nc'
)

A descriptive name of the atmospheric forcing model grid
is used in the metadata for the variables.

In [4]:
atmos_grid_name = 'HRDPS 2.5km Continental Rotated Lat/Lon'

The history metadata that will be attached to the dataset:

In [5]:
history = (
    f'[{datetime.now().strftime("%Y-%m-%d %H:%M:%S")}] '
    f'Converted to netCDF4 zlib=True dataset, and added CF-1.6 metadata '
    f'with {os.fspath(Path("tools/I_ForcingFiles/Atmos/ImproveWeightsFile.ipynb"))}'
)

**Note:** Please edit the date/time below to that of the creation of the `met_gem_weight.nc`.

In [6]:
history = '\n'.join((
    history,
    '[2023-03-13 22:38:43] '
    'Created netCDF3 classic dataset with NEMO_EastCoast/NEMO_Preparation/4_weights_ATMOS/get_weight_nemo.'
))

## 2. Open & Inspect the Input Dataset

In [7]:
src = nc.Dataset(met_gem_weight)
nc_tools.show_dataset_attrs(src)

file format: NETCDF3_CLASSIC


In [8]:
nc_tools.show_dimensions(src)

<class 'netCDF4._netCDF4.Dimension'>: name = 'x', size = 398
<class 'netCDF4._netCDF4.Dimension'>: name = 'y', size = 898
<class 'netCDF4._netCDF4.Dimension'>: name = 'lon', size = 190
<class 'netCDF4._netCDF4.Dimension'>: name = 'lat', size = 230
<class 'netCDF4._netCDF4.Dimension'>: name = 'numwgt', size = 4


In [9]:
nc_tools.show_variables(src)

dict_keys(['nav_lon', 'nav_lat', 'src01', 'wgt01', 'src02', 'wgt02', 'src03', 'wgt03', 'src04', 'wgt04'])


In [10]:
nc_tools.show_variable_attrs(src)

<class 'netCDF4._netCDF4.Variable'>
float64 nav_lon(lat, lon)
unlimited dimensions: 
current shape = (230, 190)
filling on, default _FillValue of 9.969209968386869e+36 used
<class 'netCDF4._netCDF4.Variable'>
float64 nav_lat(lat, lon)
unlimited dimensions: 
current shape = (230, 190)
filling on, default _FillValue of 9.969209968386869e+36 used
<class 'netCDF4._netCDF4.Variable'>
int32 src01(y, x)
unlimited dimensions: 
current shape = (898, 398)
filling on, default _FillValue of -2147483647 used
<class 'netCDF4._netCDF4.Variable'>
float64 wgt01(y, x)
unlimited dimensions: 
current shape = (898, 398)
filling on, default _FillValue of 9.969209968386869e+36 used
<class 'netCDF4._netCDF4.Variable'>
int32 src02(y, x)
unlimited dimensions: 
current shape = (898, 398)
filling on, default _FillValue of -2147483647 used
<class 'netCDF4._netCDF4.Variable'>
float64 wgt02(y, x)
unlimited dimensions: 
current shape = (898, 398)
filling on, default _FillValue of 9.969209968386869e+36 used
<class 'ne

## 3. Create the Output Dataset

New datasets created with the `netCDF4` library default to netCDF4 format.

In [11]:
weights = nc.Dataset(netcdf4_weight, 'w')

In [12]:
y_size, x_size = src.variables['src01'].shape
weights.createDimension('x', x_size)
weights.createDimension('y', y_size)
lat_size, lon_size = src.variables['nav_lon'].shape
weights.createDimension('lon', lon_size)
weights.createDimension('lat', lat_size)
weights.createDimension('numwgt', 4)

<class 'netCDF4._netCDF4.Dimension'>: name = 'numwgt', size = 4

Setting `zlib=True` for the variables enables Lempel-Ziv deflation
which results in a 40%-80% decrease in the file size.

In [13]:
lats = weights.createVariable('nav_lat', float, ('lat', 'lon'), zlib=True)
lons = weights.createVariable('nav_lon', float, ('lat', 'lon'), zlib=True)
src01 = weights.createVariable('src01', int, ('y', 'x'), zlib=True)
wgt01 = weights.createVariable('wgt01', float, ('y', 'x'), zlib=True)
src02 = weights.createVariable('src02', int, ('y', 'x'), zlib=True)
wgt02 = weights.createVariable('wgt02', float, ('y', 'x'), zlib=True)
src03 = weights.createVariable('src03', int, ('y', 'x'), zlib=True)
wgt03 = weights.createVariable('wgt03', float, ('y', 'x'), zlib=True)
src04 = weights.createVariable('src04', int, ('y', 'x'), zlib=True)
wgt04 = weights.createVariable('wgt04', float, ('y', 'x'), zlib=True)

Copy the latitude and longitude values,
and add their variable-level metadata:

In [14]:
lats[:] = src.variables['nav_lat'][:]
lats.units = 'degrees_north'
lats.long_name = 'Latitude'
lats.valid_range = np.array((-90.0, 90.0))
    
lons[:] = src.variables['nav_lon'][:]
lons.units = 'degrees_east'
lons.long_name = 'Longitude'
lons.valid_range = np.array((0., 360.))
    
nc_tools.show_variable_attrs(weights, 'nav_lat', 'nav_lon')

<class 'netCDF4._netCDF4.Variable'>
float64 nav_lat(lat, lon)
    units: degrees_north
    long_name: Latitude
    valid_range: [-90.  90.]
unlimited dimensions: 
current shape = (230, 190)
filling on, default _FillValue of 9.969209968386869e+36 used
<class 'netCDF4._netCDF4.Variable'>
float64 nav_lon(lat, lon)
    units: degrees_east
    long_name: Longitude
    valid_range: [  0. 360.]
unlimited dimensions: 
current shape = (230, 190)
filling on, default _FillValue of 9.969209968386869e+36 used


Copy the `src` and `wgt` variable values,
and add metadata:

In [15]:
vars = (
    (src01, wgt01), (src02, wgt02), (src03, wgt03), (src04, wgt04),
)
for i, sw in enumerate(vars):
    s, w = sw
    sname = f'src{(i+1):02d}'
    wname = f'wgt{(i+1):02d}'
    s[:] = src.variables[sname][:]
    s.units = 1
    s.long_name = f'{atmos_grid_name} Grid Index 1 (Flattened)'
    s.valid_range = np.array(
        (np.min(src.variables[sname]), np.max(src.variables[sname])))
    
    w[:] = src.variables[wname][:]
    w.units = 1
    w.long_name = f'SalishSeaCast NEMO Grid Weights for {sname}'
    w.valid_range = np.array(
        (np.min(src.variables[wname]), np.max(src.variables[wname])))

    nc_tools.show_variable_attrs(weights, sname, wname)

<class 'netCDF4._netCDF4.Variable'>
int64 src01(y, x)
    units: 1
    long_name: HRDPS 2.5km Continental Rotated Lat/Lon Grid Index 1 (Flattened)
    valid_range: [ 3893 39991]
unlimited dimensions: 
current shape = (898, 398)
filling on, default _FillValue of -9223372036854775806 used
<class 'netCDF4._netCDF4.Variable'>
float64 wgt01(y, x)
    units: 1
    long_name: SalishSeaCast NEMO Grid Weights for src01
    valid_range: [0.2503892  0.99962357]
unlimited dimensions: 
current shape = (898, 398)
filling on, default _FillValue of 9.969209968386869e+36 used
<class 'netCDF4._netCDF4.Variable'>
int64 src02(y, x)
    units: 1
    long_name: HRDPS 2.5km Continental Rotated Lat/Lon Grid Index 1 (Flattened)
    valid_range: [ 3703 40181]
unlimited dimensions: 
current shape = (898, 398)
filling on, default _FillValue of -9223372036854775806 used
<class 'netCDF4._netCDF4.Variable'>
float64 wgt02(y, x)
    units: 1
    long_name: SalishSeaCast NEMO Grid Weights for src02
    valid_range: [-2

Global dataset attributes:

In [16]:
nc_tools.init_dataset_attrs(
    weights,
    f'SalishSeaCast NEMO {atmos_grid_name} Grid Atmospheric Forcing Interpolation Weights',
    f'ImproveWeightsFile',
    os.fspath(netcdf4_weight),
)

file format: NETCDF4
Conventions: CF-1.6
title: SalishSeaCast NEMO HRDPS 2.5km Continental Rotated Lat/Lon Grid Atmospheric Forcing Interpolation Weights
institution: Dept of Earth, Ocean & Atmospheric Sciences, University of British Columbia
source: REQUIRED
references: REQUIRED
history: [2023-03-14 10:19:41] Created netCDF4 zlib=True dataset.
comment: 


not found!


In [17]:
weights.history = history
weights.source = 'NEMO_EastCoast/NEMO_Preparation/4_weights_ATMOS/get_weight_nemo'
weights.references = 'https://salishsea-meopar-docs.readthedocs.io/en/latest/code-notes/salishsea-nemo/nemo-forcing/atmospheric.html#interpolation-weights'
weights.comment = 'For use with forcing files in /results/forcing/atmospheric/continental2.5/nemo_forcing/'
nc_tools.show_dataset_attrs(weights)

file format: NETCDF4
Conventions: CF-1.6
title: SalishSeaCast NEMO HRDPS 2.5km Continental Rotated Lat/Lon Grid Atmospheric Forcing Interpolation Weights
institution: Dept of Earth, Ocean & Atmospheric Sciences, University of British Columbia
source: NEMO_EastCoast/NEMO_Preparation/4_weights_ATMOS/get_weight_nemo
references: https://salishsea-meopar-docs.readthedocs.io/en/latest/code-notes/salishsea-nemo/nemo-forcing/atmospheric.html#interpolation-weights
history: [2023-03-14 10:19:39] Converted to netCDF4 zlib=True dataset, and added CF-1.6 metadata with tools/I_ForcingFiles/Atmos/ImproveWeightsFile.ipynb
[2023-03-13 22:38:43] Created netCDF3 classic dataset with NEMO_EastCoast/NEMO_Preparation/4_weights_ATMOS/get_weight_nemo.
comment: For use with forcing files in /results/forcing/atmospheric/continental2.5/nemo_forcing/


In [18]:
nc_tools.check_dataset_attrs(weights)

In [19]:
weights.close()